In [113]:
import numpy as np
import math
from matplotlib import pyplot as plt
from ant.ant import Ant

An ant leaves its anthill in order to forage for food. It moves with the speed of 10cm per second, but it doesn’t know where to go, therefore every second it moves randomly 10cm directly north, south, east or west with equal probability.

In [114]:
ant = Ant(speed=10, seed=1)
print(f'initially\n    t: {ant.time}, loc: {ant.location}')
ant.single_move()
print(f'execute single_move\n    t: {ant.time}, loc: {ant.location}')
ant.reset()
print(f'reset\n    t: {ant.time}, loc: {ant.location}')
ant.move(n_steps=3)
print(f'move 3 steps\n    t: {ant.time}, loc: {ant.location}')

initially
    t: 0, loc: [0 0]
execute single_move
    t: 1, loc: [  0 -10]
reset
    t: 0, loc: [0 0]
move 3 steps
    t: 3, loc: [10 20]


We rephrase the behavior first.

We may choose 4 actions, east positive, east negative, north positive or north negative, with equal probability.
This is the same as consider action $A=(D, V)$, $D$ is the direction east or north, $V$ is value $10$ or $-10$.
We add time stamp $A_t=(D_t, V_t), \forall t\in\mathbb{N}$.

# Puzzle 1

If the food is located on east-west lines 20cm to the north and 20cm to the south, as well as on north-south lines 20cm to the east and 20cm to the west from the anthill, how long will it take the ant to reach it on average?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2.5, 2))
ax.axhline(-20, color='k', linewidth=3.0, linestyle="-", alpha=1, label='food')
ax.axhline(20, color='k', linewidth=3.0, linestyle="-", alpha=1)
ax.axvline(-20, color='k', linewidth=3.0, linestyle="-", alpha=1)
ax.axvline(20, color='k', linewidth=3.0, linestyle="-", alpha=1)
ax.plot([0], [0], 'o', color='C0', label='ant, t=0')
ax.legend()
plt.show()

## Semi-analytical solution


### First fix $D_t$, say we always go east or west: $D_t=$ east, $V_t = -10, 10$.
Then ant location at $t$ is $a_t = \sum_i V_i$.

Firstly it is not possible to get food when $t$ is odd.

Secondly, if we want $a_t = 20$ but $a_{i} \in [-10, 10] \forall i < t$, then $a_{t-1}=10, a_{t-2}=0, a_{t-3}=10 \ or \ -10, a_{t-4}=0$.


Then $p(a_t=20)= \frac{1}{4} p(a_{t-2}=0) = \frac{1}{8}p(a_{t-4}=0)=...=\frac{1}{2^{t/2+1}}$.
Here we omit $a_{i} \in [-10, 10] \forall i < t$ but this is taken into consideration.

As food can also be found at $a_t = -20, p(\text{food first reached at }t)=\frac{1}{2^{t/2}}$

Additionally, $p(\text{no food reached until }t)=p(a_2=0, a_4=0, ..., a_{[t/2]*2}=0)=\frac{1}{2^{[t/2]}}$.

### Add direction

What we can do here is to cluster the sequence of actions $\{ (D_i, V_i)|i=1,...,t \}$ into $E_{t_e} = \{ (D_j=\text{east}, V_j) | j=0,...,t_e \}$ and $N_{t_n} = \{ (D_j=\text{north}, V_j)|j=1,...,t_n \}$, $t_n + t_e=t$.

\begin{align*}
p(\text{food first reached along north-south at }t)
&= p(D_t=\text{north}, D_1, D_2, ...) \frac{1}{2^{t_n/2}} \frac{1}{2^{[t_e/2]}}\\
&= \frac{1}{2^t} \frac{(t-1)!}{(t_n-1)!t_e!} \frac{1}{2^{t_n/2}} \frac{1}{2^{[t_e/2]}}\\
&= \frac{1}{2^{t + t_n/2 + [t_e/2]}} \frac{(t-1)!}{(t_n-1)!t_e!}, t_n \text{ is even,} t_e \text{ can be zero,}\\

p(\text{food first reached along east-west at }t)
&= p(D_t=\text{east}, D_1, D_2, ...) \frac{1}{2^{[t_n/2]}} \frac{1}{2^{t_e/2}}\\
&= \frac{1}{2^{t + [t_n/2] + t_e/2}} \frac{(t-1)!}{t_n!(t_e-1)!}, t_e \text{ is even,} t_n \text{ can be zero}.
\end{align*}


Therefore,

\begin{align*}
t_n, t_e \text{ are even,}
p(\text{food first reached at }t)
&=
\frac{1}{2^{3t/2}} \frac{(t-1)!}{(t_n-1)!t_e!} + \frac{1}{2^{3t/2}} \frac{(t-1)!}{t_n!(t_e-1)!}\\
&=
\frac{1}{2^{3t/2}} \frac{t!}{t_n!t_e!}
,\\

t_n \text{ is even}, t_e \text{ is odd}
p(\text{food first reached at }t)
&=
\frac{1}{2^{(3t - 1)/2}} \frac{(t-1)!}{(t_n-1)!t_e!},\\

t_n \text{ is odd}, t_e \text{ is even}
p(\text{food first reached at }t)
&=
\frac{1}{2^{(3t - 1)/2}} \frac{(t-1)!}{t_n!(t_e-1)!},
\end{align*}

$\mathbb{E}[t] = \sum_t t p(\text{food first reached at }t)$.


In [115]:
def one_term(tn, te):
    t = tn + te
    if t%2 == 0:
        assert tn%2 == 0, f'{tn}, {te}'
        assert te%2 == 0, f'{tn}, {te}'
        exp_term = 2**(-3*t/2)
        fac_term = math.factorial(tn + te) / math.factorial(tn) / math.factorial(te)
    else:
        exp_term = 2**(-(3*t-1)/2)
        if tn%2 == 1:
            fac_term = math.factorial(tn + te - 1) / math.factorial(tn) / math.factorial(te - 1)
        else:
            fac_term = math.factorial(tn + te - 1) / math.factorial(tn - 1) / math.factorial(te)
    return t * fac_term * exp_term

ans = 0
for tn in range(20):
    for te in range(20):
        if tn==0 and te==0: # p=0
            continue
        if tn%2==1 and te%2==1:# p=0
            continue
        if tn%2==1 and te==0:
            continue
        if tn==0 and te%2==1:
            continue
        ans += one_term(tn, te)
print(ans)

4.499124241631264


## Solve by MC sampling

Report time step once the ant location in coordinate has inf norm equal $20$.

In [116]:
n_try = 100
reach_time = []
for _ in range(n_try):
    ant.reset()
    while True:
        ant.single_move()
        location_after_move = ant.location # 1D np.ndarray of coordinate
        food_reached = (np.linalg.norm(location_after_move, ord=np.inf)==20) # bool
        if food_reached:
            reach_time.append(ant.time)
            #print(f'get food with {ant.time} steps')
            break
print(f'average time: {np.mean(reach_time)} ')


average time: 4.39 


# Puzzle 2

What is the average time the ant will reach food if it is located only on a diagonal line passing through (10cm, 0cm) and (0cm, 10cm) points?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2.5, 2))
ax.plot([-40, 50], [50, -40], color='k', linewidth=3.0, linestyle="-", alpha=1)
ax.plot([0], [0], 'o', color='C0', label='ant, t=0')
ax.legend()
ax.set_xlim([-16, 16])
ax.set_ylim([-16, 16])
plt.show()

## Solve by MC sampling

The ant location is $(x, y)$, check time when $x + y = 10$.

In [118]:
n_try = 100
reach_time = []
for _ in range(n_try):
    ant.reset()
    while True:
        ant.single_move()
        location_after_move = ant.location # 1D np.ndarray of coordinate
        food_reached = (np.sum(location_after_move)==10) # bool
        if food_reached:
            reach_time.append(ant.time)
            #print(f'get food with {ant.time} steps')
            break
print(f'average time: {np.mean(reach_time)} ')

KeyboardInterrupt: 